<table align="left"><tr><td>
<a href="https://colab.research.google.com/github/kikim6114/blob/main/NLP2024-1/04_multilingual-ner-kikim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="코랩에서 실행하기"/></a>
</td></tr></table>

이 노트북을 코랩에서 실행하려면 Pro 버전이 필요할 수 있습니다.

In [ ]:
# 코랩을 사용하지 않으면 이 셀의 코드를 주석 처리하세요.
!git clone https://github.com/rickiepark/nlp-with-transformers.git
# %cd nlp-with-transformers
# from install import *
# install_requirements(chapter=4)

# 다중 언어 개체명 인식(NER)

- XLM-RoBERTa 사용한 NER.
- 다중언어 트랜스포머(예: BERT)는 사전훈련을 목표로 마스크드 언어모델링을 사용하지만 100개 이상의 언어로 된 텍스트에서 동시에 훈련된다.
- Zero-shot cross-lingual transfer(or learning): 한 언어에서 fine-tuning된 모델이 추가 훈련 없이 다른 언어에 적용될 수 있다.
- code-switching에도 적합.

## 데이터셋

 **E**ncoders 벤치마크 데이터셋
- WikiANN(또는 PAN-X)
  - XTREME(Cross-Lingual(**X**) **Tr**ansfer **E**valuation of **M**ultilingual) 벤치마크 데이터셋
  - 다국어 NER 데이터셋 세트
  - IOB2 형식의 LOC(위치), PER(사람), ORG(조직) 태그
  - 주석이 달린 Wikipedia 문서
  - WikiANN 코퍼스의 282개 언어 중 176개 언어를 지원
- 사용하려는 데이터셋: 스위스에서 사용되는 독일어(62.9%), 프랑스어(22.9%), 이탈리아어(8.4%), 영어(5.9%)로 구성됨

In [ ]:
import pandas as pd
toks = "Jeff Dean is a computer scientist at Google in California".split()
lbls = ["B-PER", "I-PER", "O", "O", "O", "O", "O", "B-ORG", "O", "B-LOC"]
df = pd.DataFrame(data=[toks, lbls], index=['Tokens', 'Tags'])
df

XTREME에서 PAN-X의 subset 중 하나를 load하기 위해 어떤 subset들이 제공되는지 확인

In [ ]:
from datasets import get_dataset_config_names

xtreme_subsets = get_dataset_config_names("xtreme")
print(f"XTREME 서브셋 개수: {len(xtreme_subsets)}")

범위를 좁혀 "PAN"으로 시작하는 subset을 찾자.

In [ ]:
panx_subsets = [s for s in xtreme_subsets if s.startswith("PAN")]
panx_subsets[:3]

- '.af', '.ar', '.bg'은 ISO 639-1 언어코드

In [ ]:
from datasets import load_dataset

load_dataset("xtreme", name="PAN-X.de")  # 독일어

- 사실적인 말뭉치를 만들기 위해 스위스에서 사용되는 언어 비율로 PAN-X에서 독일어(de), 프랑스어(fr), 이탈리아어(it), 영어(en) 말뭉치를 샘플링한다.
- 불균형적이지만 현실에서는 흔하다.
- 불균형적인 데이터셋을 사용해 다중 언어 애플리케이션을 다룰 때 발생하는 일반적인 상황을 시뮬레이션하고, 모든 언어에서 작동하는 모델을 만드는 방법을 알아본다.

In [ ]:
from collections import defaultdict
from datasets import DatasetDict

langs = ["de", "fr", "it", "en"]
fracs = [0.629, 0.229, 0.084, 0.059]
# 키가 없는 경우 DatasetDict를 반환합니다.
panx_ch = defaultdict(DatasetDict)

for lang, frac in zip(langs, fracs):
    # 다국어 말뭉치를 로드합니다.
    ds = load_dataset("xtreme", name=f"PAN-X.{lang}")
    # 각 분할을 언어 비율에 따라 다운샘플링하고 섞습니다.
    for split in ds:
        panx_ch[lang][split] = (
            ds[split]
            .shuffle(seed=0)
            .select(range(int(frac * ds[split].num_rows))))

In [ ]:
import pandas as pd

pd.DataFrame({lang: [panx_ch[lang]["train"].num_rows] for lang in langs},
             index=["Number of training examples"])

In [ ]:
element = panx_ch["de"]["train"][0]
for key, value in element.items():
    print(f"{key}: {value}")

NER tag를 번호 대신 태그명으로 표시해보자

In [ ]:
for key, value in panx_ch["de"]["train"].features.items():
    print(f"{key}: {value}")

In [ ]:
tags = panx_ch["de"]["train"].features["ner_tags"].feature
print(tags)

태그 읽기가 편하도록 "ner_tags_str" 열을 추가한다

In [ ]:
def create_tag_names(batch):
    return {"ner_tags_str": [tags.int2str(idx) for idx in batch["ner_tags"]]}

panx_de = panx_ch["de"].map(create_tag_names)

In [ ]:
de_example = panx_de["train"][0]
pd.DataFrame([de_example["tokens"], de_example["ner_tags_str"]],
['Tokens', 'Tags'])

태그가 불균형하게 부여되었는지 여부를 알기 위해 개체명 빈도를 계산해보자.

In [ ]:
from collections import Counter

split2freqs = defaultdict(Counter)
for split, dataset in panx_de.items():
    for row in dataset["ner_tags_str"]:
        for tag in row:
            if tag.startswith("B"):
                tag_type = tag.split("-")[1]
                split2freqs[split][tag_type] += 1
pd.DataFrame.from_dict(split2freqs, orient="index")

## 다중 언어 트랜스포머

다중 언어 트랜스포머의 훈련에서, 언어의 차이에 대한 정보를 명시적으로 제공하지 않아도 구축된 언어표현이 여러 언어의 다양한 후속 작업에 쉽게 일반화된다는 점은 놀라운 특징이다.
### 다중 언어 트랜스포머의 평가 방법
**en**  
영어 훈련데이터에서 미세조정(fine-tuning)한 다음에 각 언어의 테스트셋에서 평가.  
**each**  
언어별 성능을 측정하기 위해 단일 언어의 테스트셋에서 미세조정하고 평가.  
**all**  
모든 훈련데이터에서 미세조정하고 각 언어의 테스트셋에서 평가.
### 평가 모델
- 초기 다중 언어 트랜스포머는 `mBERT` 였으나 나중에 `XLM-RoBERTa(약칭 XLM-R)`로 대체됨
- 사전훈련 방식이 단일 언어모델인 RoBERTa와 동일
- XLM-R은 XLM에서 사용하는 언어 임베딩을 제거하고 SentencePiece를 사용해 원시 텍스트를 직접 토큰화함.
- XLM의 토큰 어휘사전 크기: 55,000
- XLM-R의 토큰 어휘사전 크기: 250,000
- XLM-R은 다국어 NLU 작업에 잘 맞는 모델이다.

## XLM-R 토큰화
- WordPiece 토크나이저 대신 100개 언어의 텍스트에서 훈련된 SentencePiece 토크나이저를 사용.

In [ ]:
from transformers import AutoTokenizer

bert_model_name = "bert-base-cased"
xlmr_model_name = "xlm-roberta-base"
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
xlmr_tokenizer = AutoTokenizer.from_pretrained(xlmr_model_name)

각 모델이 사전훈련 동안 사용하는 특수 토큰을 확인해보자

In [ ]:
text = "Jack Sparrow loves New York!"
bert_tokens = bert_tokenizer(text).tokens()
xlmr_tokens = xlmr_tokenizer(text).tokens()

In [ ]:
df = pd.DataFrame([bert_tokens, xlmr_tokens], index=["BERT", "XLM-R"])
df

### 토큰화 파이프라인

<img src="images/chapter04_tokenizer-pipeline.png" id="tokenizer-pipeline" width="500" style="margin-left: auto; margin-right: auto">
<p style="text-align: center;">Figure 4-1. 토큰화 파이프라인</p>

#### 정규화
- 소문자화
- 유니코드 정규화
- 불용어 제거
#### 사전토큰화
- 공백과 구두점 기반 단어 분리 (가능한 언어인 경우)
- 아니면, 언어별 사전토큰화 사전 사용한 단어 분리 
#### 토크나이저 모델
- BPE
- Unigram
- WordPiece
#### 후처리
- 특수 토큰 추가(
- 예: 분류 토큰 [CLS], 분할 토큰 [SEP], 시작 <s>, 끝 </s> 등

### SentencePiece 토크나이저

- 유니그램 기반의 토크나이저
- 공백이나 구두점이 없어도 가능
- 다국어 처리에 유용
- 공백 문자가 `U+2581` 또는 `_`에 할당됨
- 토큰화된 텍스트에 공백을 보존하므로 원시 텍스트를 정확히 복원

In [ ]:
# 토큰 시퀀스를 문장으로 복원하는 예
"".join(xlmr_tokens).replace(u"\u2581", " ")

## 개체명 인식을 위한 트랜스포머

<img src="images/chapter04_clf-architecture.png" id="clf-architectur" width="500" style="margin-left: auto; margin-right: auto">
<p style="text-align: center;">Figure 4-2. 시퀀스 분류를 위해 인코더 기반 트랜스포머를 미세조정하기</p>

<img src="images/chapter04_ner-architecture.png" id="clf-architectur" width="500" style="margin-left: auto; margin-right: auto">
<p style="text-align: center;">Figure 4-3. 개체명 인식을 위해 인코더 기반 트랜스포머를 미세조정하기</p>

#### 부분단어의 처리: `##ista` (from BERT 논문)
- `Christa` &rarr; `Chr` + `##ista`
- `Chr` &rarr; `B-PER`
- `##ista` &rarr; `I-PER` 또는 `B-PER` 또는 `무엇?`
- (답) 무시, 즉 `##ista` &rarr; `IGN`
- 후처리 단계에서 두 부분단어를 이어 붙일 때 첫번째 부분단어의 예측 레이블을 후속 부분단어로 쉽게 전파할 수 있다. 
- `##ista`에 `B-PER`을 할당하면 **IOB2 형식을 위반**하게 됨

## 트랜스포머 모델 클래스

- Transformers는 아키텍처 또는 task 마다 전용 class를 제공함.  
- 클래스명 형식
  - &#60;ModelName&#62;For&#60;Task&#62;
  - AutoModelFor&#60;Task&#62;  
- 트랜스포머는 기존 모델을 특정 task에 맞춰 쉽게 확장 가능하도록 설계됨.
  - 사전훈련된 모델에서 가중치를 load하고 task에 특화된 helper 함수를 사용한다. 

### 바디와 헤드

<img src="images/chapter04_bert-body-head.png" id="clf-architectur" width="200" style="margin-left: auto; margin-right: auto">
<p style="text-align: center;">Figure 4-4. BERTModel 클래스는 모델의 body만 포함하고 BertFor&#60;Task&#62; 클래스는 body와 task 전용 head를 연결한 것이다.</p>

#### Body와 Head 예
- Body: `BertModel`, `GPT2Model`
- Body + Head: `BertForMaskedLM`, `BertForSequenceClassification`

### 토큰 분류를 위한 사용자 정의 모델(토큰 분류 헤드) 만들기

In [ ]:
import torch.nn as nn
from transformers import XLMRobertaConfig
from transformers.modeling_outputs import TokenClassifierOutput
from transformers.models.roberta.modeling_roberta import RobertaModel
from transformers.models.roberta.modeling_roberta import RobertaPreTrainedModel

class XLMRobertaForTokenClassification(RobertaPreTrainedModel):
    config_class = XLMRobertaConfig  

    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        # 모델 바디를 로드합니다.
        self.roberta = RobertaModel(config, add_pooling_layer=False)
        # 토큰 분류 헤드를 준비합니다.
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)
        # 가중치를 로드하고 초기화합니다.
        self.init_weights()

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None,
                labels=None, **kwargs):
        # 모델 바디를 사용해 인코더 표현을 얻습니다.
        outputs = self.roberta(input_ids, attention_mask=attention_mask,
                               token_type_ids=token_type_ids, **kwargs)
        # 인코더 표현을 헤드에 통과시킵니다.
        sequence_output = self.dropout(outputs[0])
        logits = self.classifier(sequence_output)
        # 손실을 계산합니다.
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        # 모델 출력 객체를 반환합니다.
        return TokenClassifierOutput(loss=loss, logits=logits,
                                     hidden_states=outputs.hidden_states,
                                     attentions=outputs.attentions)

- `config_class` : `RobertaPreTrainedModel`의 부모 클래스 `PreTrainedModel`의 클래스 변수
- `super().__init__(config)` : `RobertaPreTrainedModel` 클래스의 초기화 함수 호출
- `self.dropout`, `self.classifier` : 분류 head
- `add_pooling_layer=False` : 모든 hidden state 반환
- `self.init_weights()` : `RobertaPreTrainedModel` 클래스에서 상속된 `init_weights()`로 가중치 초기화 (Body는 사전훈련된 가중치로, head는 랜덤하게)

### 사용자 정의 모델 로드하기

In [ ]:
index2tag = {idx: tag for idx, tag in enumerate(tags.names)}
tag2index = {tag: idx for idx, tag in enumerate(tags.names)}

In [ ]:
from transformers import AutoConfig

xlmr_config = AutoConfig.from_pretrained(xlmr_model_name,
                                         num_labels=tags.num_classes,
                                         id2label=index2tag, label2id=tag2index)

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
xlmr_model = (XLMRobertaForTokenClassification
              .from_pretrained(xlmr_model_name, config=xlmr_config)
              .to(device))

토크나이저와 모델을 바르게 초기화했는지 체크

In [ ]:
input_ids = xlmr_tokenizer.encode(text, return_tensors="pt")
pd.DataFrame([xlmr_tokens, input_ids[0].numpy()], index=["Tokens", "Input IDs"])

In [ ]:
outputs = xlmr_model(input_ids.to(device)).logits
predictions = torch.argmax(outputs, dim=-1)
print(f"시퀀스에 있는 토큰 개수: {len(xlmr_tokens)}")  # num_tokens
print(f"출력 크기: {outputs.shape}")  # (batch_size, num_tokens, num_tags)

In [ ]:
preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
pd.DataFrame([xlmr_tokens, preds], index=["Tokens", "Tags"])

- 토큰 분류 층(head)은 초기에 랜덤한 가중치를 가지므로 아직 성능이 나쁘다.
- 레이블링된 데이터로 fine-tuning해서 결과를 더 좋게 만들어야 하겠다.
- 앞의 과정을 헬퍼 함수로 만들자.

In [ ]:
def tag_text(text, tags, model, tokenizer):
    # 토큰을 준비합니다.
    tokens = tokenizer(text).tokens()
    # 시퀀스를 ID로 인코딩합니다.
    input_ids = xlmr_tokenizer(text, return_tensors="pt").input_ids.to(device)
    # 가능한 일곱 개의 클래스에 대한 분포를 예측합니다.
    outputs = model(input_ids)[0]
    # 토큰마다 가장 가능성 있는 클래스를 argmax로 구합니다.
    predictions = torch.argmax(outputs, dim=2)
    # 데이터프레임으로 변환합니다.
    preds = [tags.names[p] for p in predictions[0].cpu().numpy()]
    return pd.DataFrame([tokens, preds], index=["Tokens", "Tags"])

## NER 작업을 위해 텍스트 토큰화하기

In [ ]:
words, labels = de_example["tokens"], de_example["ner_tags"]

In [ ]:
tokenized_input = xlmr_tokenizer(de_example["tokens"], is_split_into_words=True)
tokens = xlmr_tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [ ]:
pd.DataFrame([tokens], index=["Tokens"])

- `Einwohnern`이 `▁Einwohner`와 `n`로 분리됨
- 첫번째 부분단어만 B-LOC 레이블과 연결어야 한다.
- `word_ids()` 메서드를 사용해서 이 작업에 도움을 받을 수 있다.
- 무시해야 할 부분단어 레이블 ID를 -100로 할당하여 Masking 한다.
- torch.nn.CrossEntropyLoss의 `ignore_index` 속성의 기본값이 -100 이다.

In [ ]:
word_ids = tokenized_input.word_ids()
pd.DataFrame([tokens, word_ids], index=["Tokens", "Word IDs"])

In [ ]:
previous_word_idx = None
label_ids = []

for word_idx in word_ids:
    if word_idx is None or word_idx == previous_word_idx:
        label_ids.append(-100)
    elif word_idx != previous_word_idx:
        label_ids.append(labels[word_idx])
    previous_word_idx = word_idx

labels = [index2tag[l] if l != -100 else "IGN" for l in label_ids]
index = ["Tokens", "Word IDs", "Label IDs", "Labels"]

pd.DataFrame([tokens, word_ids, label_ids, labels], index=index)

In [ ]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = xlmr_tokenizer(examples["tokens"], truncation=True,
                                      is_split_into_words=True)
    labels = []
    for idx, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=idx)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None or word_idx == previous_word_idx:
                label_ids.append(-100)
            else:
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
def encode_panx_dataset(corpus):
    return corpus.map(tokenize_and_align_labels, batched=True,
                      remove_columns=['langs', 'ner_tags', 'tokens'])

- 이 함수를 `DatasetDict` 객체에 적용하면 분할마다 인코딩된 Dataset 객체를 얻는다.

In [ ]:
panx_de_encoded = encode_panx_dataset(panx_ch["de"])

In [ ]:
panx_de_encoded

## 성능 측정

- 텍스트 분류모델 평가와 비슷하지만, NER에서는 한 예측이 정확하다고 판단하기 위해서는 한 개체명에 있는 모든 단어가 올바르게 예측되어야 한다.
- 이러한 문제를 위해 개발된 것이 `seqeval` 이다.

In [ ]:
from seqeval.metrics import classification_report

y_true = [["O", "O", "O", "B-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
y_pred = [["O", "O", "B-MISC", "I-MISC", "I-MISC", "I-MISC", "O"],
          ["B-PER", "I-PER", "O"]]
print(classification_report(y_true, y_pred))

- `seqeval`을 이용하기 위해, 연속된 부분단어의 레이블 ID를 무시하는 함수를 만든다.

In [ ]:
import numpy as np

def align_predictions(predictions, label_ids):
    preds = np.argmax(predictions, axis=2)
    batch_size, seq_len = preds.shape
    labels_list, preds_list = [], []

    for batch_idx in range(batch_size):
        example_labels, example_preds = [], []
        for seq_idx in range(seq_len):
            # 레이블 IDs = -100 무시
            if label_ids[batch_idx, seq_idx] != -100:
                example_labels.append(index2tag[label_ids[batch_idx][seq_idx]])
                example_preds.append(index2tag[preds[batch_idx][seq_idx]])

        labels_list.append(example_labels)
        preds_list.append(example_preds)

    return preds_list, labels_list

## XLM-RoBERTa 미세 튜닝하기

Fine-tuning 준비 완료! 어떻게 미세조정(fine-tuning)할 것인가?
- PAN-X의 독일어 subset에 base model을 fine-tuning한 다음
- 프랑스어, 이탈리아어 및 영어에서 제로샷 교차 언어 성능을 평가한다.

**훈련 속성을 정의:**    

In [ ]:
from transformers import TrainingArguments

num_epochs = 3
# 코랩에서 GPU 메모리 부족 에러가 나는 경우 batch_size를 16으로 줄여 주세요.
batch_size = 24  # 16
logging_steps = len(panx_de_encoded["train"]) // batch_size
model_name = f"{xlmr_model_name}-finetuned-panx-de"
training_args = TrainingArguments(
    output_dir=model_name, log_level="error", num_train_epochs=num_epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size, evaluation_strategy="epoch",
    save_steps=1e6, weight_decay=0.01, disable_tqdm=False,
    logging_steps=logging_steps, push_to_hub=True)

훈련 속도를 높이기 위해 다음 매개변수 값들을 크게 설정함
- weight_decay 기본값 = 0
- save_steps 기본값 = 500

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from seqeval.metrics import f1_score

def compute_metrics(eval_pred):
    y_pred, y_true = align_predictions(eval_pred.predictions,
                                       eval_pred.label_ids)
    return {"f1": f1_score(y_true, y_pred)}

Data collator는 batch에서 가장 큰 시퀀스 길이에 맞춰 padding 한다.
- 매개변수 `label_pad_token_id=-100`를 사용하여 레이블 시퀀스를 `-100`으로 패딩해서 pytorch 손실함수가 무시하도록 만든다.

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(xlmr_tokenizer)

- 몇 개의 모델을 훈련하므로, Trainer를 위해 매번 새 모델을 만들지 않도록 model_init() 함수를 만든다.
- train() 메서드를 호출할 때 이 함수가 호출되어 훈련되지 않은 모델을 로드하게 된다.

In [ ]:
def model_init():
    return (XLMRobertaForTokenClassification
            .from_pretrained(xlmr_model_name, config=xlmr_config)
            .to(device))

In [ ]:
%env TOKENIZERS_PARALLELISM=false

In [ ]:
from transformers import Trainer

trainer = Trainer(model_init=model_init, args=training_args,
                  data_collator=data_collator, compute_metrics=compute_metrics,
                  train_dataset=panx_de_encoded["train"],
                  eval_dataset=panx_de_encoded["validation"],
                  tokenizer=xlmr_tokenizer)

In [ ]:
trainer.train()
trainer.push_to_hub(commit_message="Training completed!")

In [ ]:
df = pd.DataFrame(trainer.state.log_history)[['epoch','loss' ,'eval_loss', 'eval_f1']]
df = df.rename(columns={"epoch":"Epoch","loss": "Training Loss", "eval_loss": "Validation Loss", "eval_f1":"F1"})
df['Epoch'] = df["Epoch"].apply(lambda x: round(x))
df['Training Loss'] = df["Training Loss"].ffill()
df[['Validation Loss', 'F1']] = df[['Validation Loss', 'F1']].bfill().ffill()
df.drop_duplicates()

In [ ]:
text_de = "Jeff Dean ist ein Informatiker bei Google in Kalifornien"
tag_text(text_de, tags, trainer.model, xlmr_tokenizer)

## 오류 분석

모델이 잘 작동하는 것처럼 보이지만 심각한 결함이 있어 훈련에 실패하는 경우들:
- 우연히 너무 많은 토큰을 마스킹하고 일부 레이블도 마스킹하여 제대로 훈련이 되는 것처럼 손실이 감소.
- `compute_metrics()` 함수에 실제 성능을 과대평가하는 버그가 있을 수 있다.
- NER에 0 클래스 또는 0 개체명이 일반 클래스처럼 포함될 때가 있다. 압도적인 다수 클래스이므로 정확도와 F1-score가 크게 왜곡된다.

- 모델의 성능이 기대보다 낮을 때, 오류를 살펴보면 유용한 통찰을 얻고 코드만 봐서는 찾기 힘든 버그를 발견할 수 있다.
- 모델이 잘 작동하더라도 오류 분석은 모델의 강점과 약점을 파악하는 유용한 도구다.

**강력한**한가지 방법으로, 손실이 가장 큰 샘플을 살펴보자.
- 샘플 시퀀스의 토큰마다 손실을 계산한다.

In [ ]:
from torch.nn.functional import cross_entropy

def forward_pass_with_label(batch):
    # 리스트의 딕셔너리를 데이터 콜레이터에 적합한 딕셔너리의 리스트로 변환합니다.
    features = [dict(zip(batch, t)) for t in zip(*batch.values())]
    # 입력과 레이블을 패딩하고 모든 텐서를 장치에 배치합니다.
    batch = data_collator(features)
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)
    with torch.no_grad():
        # 데이터를 모델에 전달합니다.
        output = trainer.model(input_ids, attention_mask)
        # Logit.size: [batch_size, sequence_length, classes]
        # 마지막 축을 따라 가장 큰 로짓 값을 가진 클래스를 선택합니다.
        predicted_label = torch.argmax(output.logits, axis=-1).cpu().numpy()
    # 배치 차원을 펼친 다음 토큰마다 손실을 계산합니다.
    loss = cross_entropy(output.logits.view(-1, 7),
                         labels.view(-1), reduction="none")
    # 배치 차원을 다시 만들고 넘파이 배열로 변환합니다.
    loss = loss.view(len(input_ids), -1).cpu().numpy()

    return {"loss":loss, "predicted_label": predicted_label}

이 함수를 map() 메서드를 사용해 전체 validation set에 적용

In [ ]:
valid_set = panx_de_encoded["validation"]
valid_set = valid_set.map(forward_pass_with_label, batched=True, batch_size=32)
df = valid_set.to_pandas()

알아보기 쉽게 토근과 레이블을 다시 문자열로 변환
- '-100'을 `IGN`로
- loss와 predicted_label을 입력과 같은 길이로 자름(패딩을 없앰) 

In [ ]:
index2tag[-100] = "IGN"
df["input_tokens"] = df["input_ids"].apply(
    lambda x: xlmr_tokenizer.convert_ids_to_tokens(x))
df["predicted_label"] = df["predicted_label"].apply(
    lambda x: [index2tag[i] for i in x])
df["labels"] = df["labels"].apply(
    lambda x: [index2tag[i] for i in x])
df['loss'] = df.apply(
    lambda x: x['loss'][:len(x['input_ids'])], axis=1)
df['predicted_label'] = df.apply(
    lambda x: x['predicted_label'][:len(x['input_ids'])], axis=1)
df.head(1)

pd.Series.explode() : 리스트에 있는 각 원소를 하나의 행으로 만들 수 있다.

In [ ]:
df_tokens = df.apply(pd.Series.explode)
df_tokens = df_tokens.query("labels != 'IGN'")
df_tokens["loss"] = df_tokens["loss"].astype(float).round(2)
df_tokens.head(7)

In [ ]:
(
    df_tokens.groupby("input_tokens")[["loss"]]
    .agg(["count", "mean", "sum"])
    .droplevel(level=0, axis=1)  # 멀티 컬럼을 삭제합니다.
    .sort_values(by="sum", ascending=False)
    .reset_index()
    .round(2)
    .head(10)
    .T
)

레이블 ID로 grouping해서 각 클래스에 대한 손실을 볼 수도 있다.

In [ ]:
(
    df_tokens.groupby("labels")[["loss"]]
    .agg(["count", "mean", "sum"])
    .droplevel(level=0, axis=1)
    .sort_values(by="mean", ascending=False)
    .reset_index()
    .round(2)
    .T
)

B-ORG의 손실이 가장 높다. 
- 모델이 조직 이름의 시작 부분을 결정하기 어렵다는 의미
- B-ORG와 I-ORG가 혼동되는 경우가 많아 보인다.

더 세분화하기 위해 토큰 분류의 오차행렬을 그려보자.

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

def plot_confusion_matrix(y_preds, y_true, labels):
    cm = confusion_matrix(y_true, y_preds, normalize="true")
    fig, ax = plt.subplots(figsize=(6, 6))
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap="Blues", values_format=".2f", ax=ax, colorbar=False)
    plt.title("Normalized confusion matrix")
    plt.show()

In [ ]:
plot_confusion_matrix(df_tokens["labels"], df_tokens["predicted_label"],
                      tags.names)

- 그래프를 보면 모델은 B-ORG와 I-ORG를 가장 많이 혼동한다
- 대각선 부분을 보면, 나머지 개체명은 잘 분류한다는 것을 알 수 있다.

이번에는, 높은 손실을 내는 시퀀스는 무엇일까?   
우선 토큰 시퀀스, 레이블, 손실을 출력하는 함수를 만들자.

In [ ]:
def get_samples(df):
    for _, row in df.iterrows():
        labels, preds, tokens, losses = [], [], [], []
        for i, mask in enumerate(row["attention_mask"]):
            if i not in {0, len(row["attention_mask"])}:
                labels.append(row["labels"][i])
                preds.append(row["predicted_label"][i])
                tokens.append(row["input_tokens"][i])
                losses.append(f"{row['loss'][i]:.2f}")
        df_tmp = pd.DataFrame({"tokens": tokens, "labels": labels,
                               "preds": preds, "losses": losses}).T
        yield df_tmp

df["total_loss"] = df["loss"].apply(sum)
df_tmp = df.sort_values(by="total_loss", ascending=False).head(3)

for sample in get_samples(df_tmp):
    display(sample)

- 마지막 샘플의 레이블에 문제가 있어보인다.
- 첫번째 샘플도 레이블 문제가 있어보인다.
- PAN-X 데이터셋의 레이블이 자동 생성된 것이다 -> 실버 스탠다드
- 괄호와 슬래시의 손실이 비교적 높다. 시작 괄호가 있는 문장을 조금 더 보자.

In [ ]:
df_tmp = df.loc[df["input_tokens"].apply(lambda x: u"\u2581(" in x)].head(2)
for sample in get_samples(df_tmp):
    display(sample)

## 교차 언어 전이

In [ ]:
def get_f1_score(trainer, dataset):
    return trainer.predict(dataset).metrics["test_f1"]

테스트셋의 성능을 평가하고 테스트 결과를 기록해둔다.
도

In [ ]:
f1_scores = defaultdict(dict)
f1_scores["de"]["de"] = get_f1_score(trainer, panx_de_encoded["test"])
print(f"[de] 데이터셋에서 [de] 모델의 F1-점수: {f1_scores['de']['de']:.3f}")

독일어에서 미세조정한 모델을 프랑스어에서 평가해보자

In [ ]:
text_fr = "Jeff Dean est informaticien chez Google en Californie"
tag_text(text_fr, tags, trainer.model, xlmr_tokenizer)

In [ ]:
def evaluate_lang_performance(lang, trainer):
    panx_ds = encode_panx_dataset(panx_ch[lang])
    return get_f1_score(trainer, panx_ds["test"])

In [ ]:
f1_scores["de"]["fr"] = evaluate_lang_performance("fr", trainer)
print(f"[fr] 데이터셋에서 [de] 모델의 F1-점수: {f1_scores['de']['fr']:.3f}")

F1 score가 좀 낮아졌지만 이 모델은 레이블된 프랑스어를 한 번도 본적이 없다!

In [ ]:
f1_scores["de"]["it"] = evaluate_lang_performance("it", trainer)
print(f"[it] 데이터셋에서 [de] 모델의 F1-점수: {f1_scores['de']['it']:.3f}")

In [ ]:
f1_scores["de"]["en"] = evaluate_lang_performance("en", trainer)
print(f"[en] 데이터셋에서 [de] 모델의 F1-점수: {f1_scores['de']['en']:.3f}")

### 제로 샷 전이가 유용할 때

- 앞에서는, 독일어 코퍼스에서 미세조정한 XLM-R로 다른 언어(예 프랑스어)에서 평가하여 비교적 좋은 점수를 얻었다.(교차 언어 전이)
- 프랑스어에서 미세튜닝하여 프랑스어에서 테스트하면 어떨까? 훈련셋의 크기를 증가시켜 가면서 교차 언어 전이보다 좋아지는 훈련셋의 크기를 구해본다.
- 레이블링된 데이터셋을 더 많이 수집해야 하는지 판단하는 데 유용한 전략이다.

In [ ]:
def train_on_subset(dataset, num_samples):
    train_ds = dataset["train"].shuffle(seed=42).select(range(num_samples))
    valid_ds = dataset["validation"]
    test_ds = dataset["test"]
    training_args.logging_steps = len(train_ds) // batch_size

    trainer = Trainer(model_init=model_init, args=training_args,
        data_collator=data_collator, compute_metrics=compute_metrics,
        train_dataset=train_ds, eval_dataset=valid_ds, tokenizer=xlmr_tokenizer)
    trainer.train()
    if training_args.push_to_hub:
        trainer.push_to_hub(commit_message="Training completed!")

    f1_score = get_f1_score(trainer, test_ds)
    return pd.DataFrame.from_dict(
        {"num_samples": [len(train_ds)], "f1_score": [f1_score]})

독일어 말뭉치처럼, 프랑스어 말뭉치를 입력ID, 어텐션마스크, 레이블ID로 인코딩한다.

In [ ]:
panx_fr_encoded = encode_panx_dataset(panx_ch["fr"])

250개의 작은 훈련셋에서 이 함수를 실행한다.

In [ ]:
training_args.push_to_hub = False
metrics_df = train_on_subset(panx_fr_encoded, 250)
metrics_df

250개 일대는 독일어에서 제로샷 전이로 얻은 결과보다 크게 떨어진다.

In [ ]:
for num_samples in [500, 1000, 2000, 4000]:
    metrics_df = pd.concat([
        metrics_df,
        train_on_subset(panx_fr_encoded, num_samples)])

그래프로 그려보자.

In [ ]:
fig, ax = plt.subplots()
ax.axhline(f1_scores["de"]["fr"], ls="--", color="r")
metrics_df.set_index("num_samples").plot(ax=ax)
plt.legend(["Zero-shot from de", "Fine-tuned on fr"], loc="lower right")
plt.ylim((0, 1))
plt.xlabel("Number of Training Samples")
plt.ylabel("F1 Score")
plt.show()

그래프에서 보면, 훈련 샘플이 약 750개일 때까지 제로샷 전이가 앞선다.

### 다국어에서 동시에 미세 튜닝하기

- 제로삿 교차언어 전이에서 성능 저하를 줄리는 방법 하나는 다국어에서 동시에 미세조정하는 것이다.
- 우선 독일어와 프랑스어 말뭉치를 합쳐보자.

In [ ]:
from datasets import concatenate_datasets

def concatenate_splits(corpora):
    multi_corpus = DatasetDict()
    for split in corpora[0].keys():
        multi_corpus[split] = concatenate_datasets(
            [corpus[split] for corpus in corpora]).shuffle(seed=42)
    return multi_corpus

In [ ]:
panx_de_fr_encoded = concatenate_splits([panx_de_encoded, panx_fr_encoded])

In [ ]:
training_args.logging_steps = len(panx_de_fr_encoded["train"]) // batch_size
training_args.push_to_hub = True
training_args.output_dir = "xlm-roberta-base-finetuned-panx-de-fr"

trainer = Trainer(model_init=model_init, args=training_args,
    data_collator=data_collator, compute_metrics=compute_metrics,
    tokenizer=xlmr_tokenizer, train_dataset=panx_de_fr_encoded["train"],
    eval_dataset=panx_de_fr_encoded["validation"])

trainer.train()
trainer.push_to_hub(commit_message="Training completed!")

In [ ]:
for lang in langs:
    f1 = evaluate_lang_performance(lang, trainer)
    print(f"[{lang}] 데이터셋에서 [de-fr] 모델의 F1-점수: {f1:.3f}")

- 프랑스어 테스트 성능이 훨씬 좋아져서 독일어 테스트 성능과 비슷해졌다.
- 이탈리아어와 영어 성능도 약 10% 좋아졌다.
- 즉, 다른 언어를 추가해도 본 적 없는 언어에서의 모델 성능이 개선된다!!

나머지 언어를 `train_on_subset()`에서 미세조정한다.

In [ ]:
corpora = [panx_de_encoded]

# 반복에서 독일어는 제외합니다.
for lang in langs[1:]:
    training_args.output_dir = f"xlm-roberta-base-finetuned-panx-{lang}"
    # 단일 언어 말뭉치에서 미세 튜닝합니다.
    ds_encoded = encode_panx_dataset(panx_ch[lang])
    metrics = train_on_subset(ds_encoded, ds_encoded["train"].num_rows)
    # 딕셔너리에 F1-점수를 모읍니다.
    f1_scores[lang][lang] = metrics["f1_score"][0]
    # 단일 언어 말뭉치를 corpora 리스트에 추가합니다.
    corpora.append(ds_encoded)

모든 언어의 분할을 합쳐서 네 언어로 구성된 다국어 말뭉치를 만든다.

In [ ]:
corpora_encoded = concatenate_splits(corpora)

In [ ]:
training_args.logging_steps = len(corpora_encoded["train"]) // batch_size
training_args.output_dir = "xlm-roberta-base-finetuned-panx-all"

trainer = Trainer(model_init=model_init, args=training_args,
    data_collator=data_collator, compute_metrics=compute_metrics,
    tokenizer=xlmr_tokenizer, train_dataset=corpora_encoded["train"],
    eval_dataset=corpora_encoded["validation"])

trainer.train()
trainer.push_to_hub(commit_message="Training completed!")

In [ ]:
for idx, lang in enumerate(langs):
    f1_scores["all"][lang] = get_f1_score(trainer, corpora[idx]["test"])

In [ ]:
scores_data = {"de": f1_scores["de"],
               "each": {lang: f1_scores[lang][lang] for lang in langs},
               "all": f1_scores["all"]}
f1_scores_df = pd.DataFrame(scores_data).T.round(4)
f1_scores_df.rename_axis(index="Fine-tune on", columns="Evaluated on",
                         inplace=True)
f1_scores_df

**결론**  
- 다중 언어 학습은 성능상 큰 이득을 제공한다. 특히 유사한 언어군에서 데이터가 부족한 언어로 교차 언어 전이를 수행할 때 큰 이득이 된다. `all` 행을 보면, 독일어, 프랑스어, 이탈리아어가 비슷한 언어라고 생각할 수 있다.
- 일본어처럼 크게 다른 언어를 다룰 때는 한 어족 내에서 교차 언어 전이를 하도록 하는 것이 좋다.

## 모델 위젯 사용하기

<img alt="A Hub widget" caption="Example of a widget on the Hugging Face Hub" src="https://github.com/rickiepark/nlp-with-transformers/blob/main/images/chapter04_ner-widget.png?raw=1" id="ner-widget"/>  

## 결론